# Анализ данных и процессов
## Лабораторная работа №3. Поток событий без очередей.
### Исходные данные.
На сервере есть n каналов передачи сообщений. Среднее время обработки сообщений τ минут. На сервер поступают сообщения в среднем количестве λ сообщений в минуту.  Параметры варианта определяются по формулам: n=3+[(i+j)/8], λ=1+i/4, τ=5/(5+j). Здесь квадратные скобки означают взятие целой части, а i,j - последние цифры зачетки.

In [1]:
import numpy as np

i, j, = 2, 3

N = 3 + round((i + j) / 8)
LAMBDA = 1 + i / 4
TAU = 5 / (5 + j)

print("Количество каналов N:", N)
print("Интенсивность потока событий λ:", LAMBDA)
print("Среднее время обслуживания τ:", TAU)

Количество каналов N: 4
Интенсивность потока событий λ: 1.5
Среднее время обслуживания τ: 0.625


### Задание 1. Определить основные характеристики сервера.
$A$ - абсолютную пропускную способность, т.е. среднее число сообщений, обрабатываемых в единицу времени;

$Q$ - относительную пропускную способность, т.е. среднюю долю пришедших сообщений, обрабатываемых системой;

$P_{отк}$ - вероятность отказа в обработке;

$k$ - среднее число занятых каналов, если сообщение получает отказ при занятости всех каналов.

In [2]:
MU = 1 / TAU
print("Скорость обработки сообщений μ:", MU)
RHO = LAMBDA / MU
print("Интенсивность потока ρ:", RHO)

Скорость обработки сообщений μ: 1.6
Интенсивность потока ρ: 0.9375


In [3]:
import math

p = [sum(RHO ** i / math.factorial(i) for i in range(N + 1)) ** -1]


for i in range(1, N + 1):
    cur_p = (RHO ** i / math.factorial(i)) * p[0]
    p.append(cur_p)
    
print(p)

[0.392700678534593, 0.3681568861261809, 0.17257354037164732, 0.053929231366139786, 0.012639663601439012]


In [4]:
p_reject = p[N]
q = 1 - p_reject
a = LAMBDA * q
k = sum(i * p[i] for i in range(1, N + 1))

print("Вероятность отказа в обработке p:", p_reject)
print(f"Относительная пропускная способность Q: {q} ({q * 100}%)")
print("Абсолютная пропускная способность A:", a)
print("Среднее количество занятых каналов k:", k)

Вероятность отказа в обработке p: 0.012639663601439012
Относительная пропускная способность Q: 0.987360336398561 (98.7360336398561%)
Абсолютная пропускная способность A: 1.4810405045978414
Среднее количество занятых каналов k: 0.925650315373651


### Задание 2. Написать программу, которая имитирует поведение сервера и вычисляет его основные характеристики.

#### Класс симулятора

In [5]:
from dataclasses import dataclass
from typing import Generator

@dataclass
class State:
    busy_channels: int
    time: float


class ServerSimulator:
    def __init__(self, time_limit: float, max_channels: int):
        self._time_limit = time_limit
        self._max_channels = max_channels

    def iter_states(self) -> Generator:
        yield (current_state := State(0, 0))

        while current_state.time < self._time_limit:
            next_arrive_delta_time = -np.log(1 - np.random.random()) / LAMBDA

            if current_state.busy_channels == 0:
                yield (current_state := State(1, current_state.time + next_arrive_delta_time))
            else:
                next_processed_delta_time = -np.log(1 - np.random.random()) / (current_state.busy_channels * MU)

                if next_arrive_delta_time < next_processed_delta_time:
                    if current_state.busy_channels < self._max_channels:
                        yield (current_state := State(current_state.busy_channels + 1,
                                                      current_state.time + next_arrive_delta_time))
                    else:
                        yield (current_state := State(current_state.busy_channels,
                                                      current_state.time + next_arrive_delta_time))

                else:
                    yield (current_state := State(current_state.busy_channels - 1,
                                                  current_state.time + next_processed_delta_time))


#### Имитация обработки событий сервером

In [6]:
import itertools

ITERS_COUNT = 10000
TIME_LIMIT = 200    

simulator = ServerSimulator(TIME_LIMIT, N)
durations = np.zeros((ITERS_COUNT, N + 1))
processed_counts = np.zeros(ITERS_COUNT)

for index in range(ITERS_COUNT):
    for previous, current in itertools.pairwise(simulator.iter_states()):
        durations[index][previous.busy_channels] += current.time - previous.time
        if current.busy_channels < previous.busy_channels:
            processed_counts[index] += 1


#### Вычисление фактических характеристик сервера

In [7]:
COEFFICIENT = TIME_LIMIT * ITERS_COUNT

p_fact = sum(durations) / COEFFICIENT
q_fact = 1 - p_fact[N]
a_fact = sum(processed_counts) / COEFFICIENT
k_fact = sum(i * p_i for i, p_i in enumerate(p_fact))

### Задание 3. Сравнить результаты.

#### Вероятности занятости каналов

In [8]:
import pandas as pd

pd.DataFrame(np.vstack((p, p_fact)).transpose(), columns=["Ожидаемые", "Фактические"])

,Ожидаемые,Фактические
0,0.392701,0.395386
1,0.368157,0.368427
2,0.172574,0.172068
3,0.053929,0.053708
4,0.012640,0.012518


#### Другие характеристики

In [9]:
pd.DataFrame(np.array([
    [q, q_fact],
    [a, a_fact],
    [k, k_fact],
]), index=["Q", "A", "k"], columns=["Ожидаемые", "Фактические"])

,Ожидаемые,Фактические
Q,0.987360,0.987482
A,1.481041,1.479421
k,0.925650,0.923759


## Вывод
В ходе выполнения лабораторной работы была спроектирована модель системы массового обслуживания на основе простейшего потока событий. Были теоретически рассчитаны ее ожидаемые характеристики. Затем была написана программа, имитирующая поведение системы, и получены фактические характеристики. Проведен сравнительный анализ ожидаемых и фактических характеристик. 